In [ ]:
import torch

# Modifying tensors

1. *View (`torch.Tensor.view`). Allows us to change the shape of the tensor, similar to what the `torch.Tensor.reshape()` method does, but with two major differences:*
    - *It doesn't make a copy of the tensor in memory. The new tensor references the same object in memory.*
    - *It only works if the tensor is **contiguous** in memory.*
        - *A tensor being contiguous means that its elements are stored sequentially in an uninterrupted block of memory (one-dimensional vector).*
    - *There are operations that can make a tensor non-contiguous, for example, transposing a tensor. In this case, this method will return an error (you can call the `torch.Tensor.contiguous()` method before calling `torch.Tensor.view()`).*
2. *Reshape (`torch.Tensor.reshape()`). Allows us to change the shape of the tensor without changing its elements. For example, changing the dimension from $3\times 2$ to $2\times 3$.*
    - *Doesn't require the tensor to be contiguous in memory. If the tensor is contiguous, it calls the `torch.Tensor.view()` method. If it's not contiguous, then it first calls the `torch.Tensor.contiguous()` method, creating a copy of the tensor in memory that is contiguous.*
    - *If we're not sure whether the tensor is contiguous or not, it's better to use `torch.Tensor.reshape()` instead of `torch.Tensor.view()`.*
3. *Concatenate. There are several options available for concatenating tensors:*
    -  *`torch.cat()`. Allows concatenating tensors along an existing dimension, resulting in a tensor with the same number of dimensions. For example, if we have two $2\times 2$ tensors, and we concatenate them using `dim=0` (in this case rows), the result will be a $4\times 2$ tensor, while if we concatenate them using `dim=1` (in this case columns), the result will be a $2\times 4$ tensor.*
        - *It's not necessary for the dimension along which we concatenate to have the same size, but the remaining dimensions must be equal.*
    - *`torch.stack()`. Allows concatenating tensors along a new dimension, resulting in a tensor with an additional dimension. For example, if we have two $2\times 2$ tensors, and we concatenate them using `dim=0`, then it returns a tensor of dimension $2\times2\times2$.*
        - *Both tensors must have the same size.*
4. *Remove/add dimensions. The `torch.squeeze()` function allows us to remove dimensions of size 1, while the `torch.unsqueeze()` function allows us to add dimensions of size 1.*
5. *Permute dimensions. With the `torch.permute()` function we can rearrange the dimensions of a tensor.*

## Views

In [47]:
X = torch.arange(0, 10)
X = X.type(torch.float32)
X

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [48]:
y = X.view(2, 5)
y

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

- *A new tensor `y` was created as a view of tensor `X`, with shape $2\times 5$. Modifying any element of tensor `y` will also affect tensor `X`, since `y` shares the same underlying memory as `X`. This behavior occurs because views reference the original tensor's data rather than creating a copy.*

In [49]:
y[:, 0] = torch.tensor([10, 20], dtype=torch.float32)
y

tensor([[10.,  1.,  2.,  3.,  4.],
        [20.,  6.,  7.,  8.,  9.]])

In [50]:
X

tensor([10.,  1.,  2.,  3.,  4., 20.,  6.,  7.,  8.,  9.])

## Reshape

In [68]:
X = torch.rand(size=(2, 3))
X

tensor([[0.3700, 0.5732, 0.2554],
        [0.2901, 0.9397, 0.7976]])

In [69]:
y = X.reshape(3, -1) # -1 means that the size of that dimension is defined automatically
y

tensor([[0.3700, 0.5732],
        [0.2554, 0.2901],
        [0.9397, 0.7976]])

- *Here, the `reshape` method gives us a view of tensor `X`, so any changes we make to `y` will also show up in `X`. That's because `X` is still stored contiguously in memory, letting `reshape` just reference the original data instead of making a copy.*

In [70]:
y[0, :] = torch.tensor([1, 1])
y

tensor([[1.0000, 1.0000],
        [0.2554, 0.2901],
        [0.9397, 0.7976]])

In [71]:
X

tensor([[1.0000, 1.0000, 0.2554],
        [0.2901, 0.9397, 0.7976]])

- *We create a new tensor `z` by transposing `X`, and then generate tensor `y` from `z`. If we modify `y`, those changes won’t affect `X`. Why? Because transposing `X` makes `z` non-contiguous in memory, so when we call `reshape()` on `z`, PyTorch creates a copy instead of a view. That’s why `y` is independent from `X`—they no longer share the same underlying data.*

In [72]:
z = X.T
z

tensor([[1.0000, 0.2901],
        [1.0000, 0.9397],
        [0.2554, 0.7976]])

In [73]:
y = z.reshape(2, -1)
y

tensor([[1.0000, 0.2901, 1.0000],
        [0.9397, 0.2554, 0.7976]])

In [74]:
y[:, 0] = torch.tensor([0, 0])
y

tensor([[0.0000, 0.2901, 1.0000],
        [0.0000, 0.2554, 0.7976]])

In [75]:
X

tensor([[1.0000, 1.0000, 0.2554],
        [0.2901, 0.9397, 0.7976]])

## Concatenate

In [77]:
tensor_A = torch.rand(size=(2, 3))
tensor_B = torch.rand(size=(2, 3))

print(f'tensor_A:\n {tensor_A}')
print(f'tensor_B:\n {tensor_B}')

tensor_A:
 tensor([[0.9063, 0.8582, 0.9965],
        [0.8017, 0.4278, 0.9973]])
tensor_B:
 tensor([[0.9701, 0.1752, 0.0940],
        [0.1659, 0.8378, 0.3756]])


- *If we use `torch.cat()`, we can concatenate two tensors along an existing dimension (maintains the size of the dimensions along which we don't concatenate). For example, if we concatenate along `dim=0`, we get a $4\times 2$ tensor (the size of the second dimension is maintained).*

In [80]:
tensor_C = torch.cat((tensor_A, tensor_B), dim=0)
print(f'Original shape: tensor_A = {tensor_A.shape} and tensor_B = {tensor_B.shape}')
print(f'Shape after concatenation: tensor_C = {tensor_C.shape}\n')
print(f'New tensor:\n {tensor_C}')

Original shape: tensor_A = torch.Size([2, 3]) and tensor_B = torch.Size([2, 3])
Shape after concatenation: tensor_C = torch.Size([4, 3])

New tensor:
 tensor([[0.9063, 0.8582, 0.9965],
        [0.8017, 0.4278, 0.9973],
        [0.9701, 0.1752, 0.0940],
        [0.1659, 0.8378, 0.3756]])


*If, on the other hand, we use dimension `dim=1`, we get a $2\times6$ tensor.*

In [81]:
tensor_C = torch.cat((tensor_A, tensor_B), dim=1)
print(f'Original shape: tensor_A = {tensor_A.shape} and tensor_B = {tensor_B.shape}')
print(f'Shape after concatenation: tensor_C = {tensor_C.shape}\n')
print(f'New tensor:\n {tensor_C}')

Original shape: tensor_A = torch.Size([2, 3]) and tensor_B = torch.Size([2, 3])
Shape after concatenation: tensor_C = torch.Size([2, 6])

New tensor:
 tensor([[0.9063, 0.8582, 0.9965, 0.9701, 0.1752, 0.0940],
        [0.8017, 0.4278, 0.9973, 0.1659, 0.8378, 0.3756]])


- *If we use `torch.stack()`, we create a new dimension. So, in this case we get a three-dimensional tensor from concatenating two two-dimensional tensors. In this case it's necessary that both tensors have the same dimensions.*

In [82]:
tensor_C = torch.stack((tensor_A, tensor_B), dim=0)
print(f'Original shape: tensor_A = {tensor_A.shape} and tensor_B = {tensor_B.shape}')
print(f'Shape after concatenation: tensor_C = {tensor_C.shape}\n')
print(f'New tensor:\n {tensor_C}')

Original shape: tensor_A = torch.Size([2, 3]) and tensor_B = torch.Size([2, 3])
Shape after concatenation: tensor_C = torch.Size([2, 2, 3])

New tensor:
 tensor([[[0.9063, 0.8582, 0.9965],
         [0.8017, 0.4278, 0.9973]],

        [[0.9701, 0.1752, 0.0940],
         [0.1659, 0.8378, 0.3756]]])


## Remove/add dimensions

- *The `torch.squeeze()` function lets you drop dimensions of size 1 from a tensor. You can use the `dim` argument to specify exactly which dimensions to remove.*
- *Remember, the result is a view (`torch.Tensor.view`), so it shares memory with the original tensor—any changes you make will affect both.*

In [93]:
X = torch.rand(size=(1, 2, 3))
X

tensor([[[0.1866, 0.0648, 0.6463],
         [0.6808, 0.5452, 0.9616]]])

- *We dropped the first dimension (size 1), so now `tensor_X_squeezed` is a $2\times 3$ matrix. This makes the tensor easier to work with, since those singleton dimensions often just get in the way.*

In [87]:
tensor_X_squeezed = torch.squeeze(X)
print(f'Original shape: {X.shape}')
print(f'Shape after squeezing: {tensor_X_squeezed.shape}\n')
print(f'New tensor:\n {tensor_X_squeezed}')

Original shape: torch.Size([1, 2, 3])
Shape after squeezing: torch.Size([2, 3])

New tensor:
 tensor([[0.9786, 0.7882, 0.6897],
        [0.1621, 0.8248, 0.8890]])


- *On the other hand, `torch.unsqueeze()` lets you add a dimension wherever you need it—just specify which one. In the example below, we’re putting back the dimension we squeezed out earlier. This is super handy for prepping tensors for operations that expect a certain shape.*

In [92]:
tensor_X_unsqueezed = torch.unsqueeze(tensor_X_squeezed, dim=0)
print(f'Shape squeezed tensor: {tensor_X_squeezed.shape}')
print(f'Shape after unsqueezing: {tensor_X_unsqueezed.shape}\n')
print(f'New tensor:\n {tensor_X_unsqueezed}')

Shape squeezed tensor: torch.Size([2, 3])
Shape after unsqueezing: torch.Size([2, 1, 3])

New tensor:
 tensor([[[0.9786, 0.7882, 0.6897]],

        [[0.1621, 0.8248, 0.8890]]])


## Permute dimensions

- *With `torch.permute()`, you can shuffle the order of a tensor’s dimensions however you want. For example, suppose you have a $1024 \times 1024 \times 3$ tensor—think of it as an image, where the first two dimensions are height and width, and the last one is the color channels (RGB). If you need to move the channel dimension to the front (so it’s $3 \times 1024 \times 1024$), just specify the new order of axes. This is super useful for prepping data for deep learning models that expect a specific format.*

In [94]:
image = torch.rand(size=(1024, 1024, 3))
image

tensor([[[0.9683, 0.9730, 0.2627],
         [0.8097, 0.8765, 0.0550],
         [0.6498, 0.4069, 0.7136],
         ...,
         [0.3865, 0.8466, 0.9223],
         [0.7619, 0.6037, 0.4737],
         [0.6288, 0.7732, 0.5016]],

        [[0.7348, 0.9181, 0.7637],
         [0.3168, 0.6106, 0.4738],
         [0.4297, 0.4724, 0.0670],
         ...,
         [0.6849, 0.3207, 0.5561],
         [0.5420, 0.3546, 0.4909],
         [0.5327, 0.2005, 0.7087]],

        [[0.5963, 0.4694, 0.3344],
         [0.8822, 0.2125, 0.4717],
         [0.0236, 0.4680, 0.9261],
         ...,
         [0.5447, 0.1777, 0.9908],
         [0.6046, 0.7683, 0.6753],
         [0.4742, 0.0015, 0.0772]],

        ...,

        [[0.8567, 0.7679, 0.7789],
         [0.4698, 0.0566, 0.6014],
         [0.8258, 0.4664, 0.5827],
         ...,
         [0.7187, 0.1043, 0.7711],
         [0.3493, 0.7985, 0.1038],
         [0.0346, 0.3198, 0.4269]],

        [[0.6278, 0.5091, 0.4229],
         [0.0168, 0.8149, 0.7544],
         [0.

- *To permute tensor dimensions, just list the axes in the order you want them to appear in the new tensor. Here, we’re moving the third dimension (channels, `dim=2`) to the front, followed by the first (`dim=0`) and second (`dim=1`).*

In [95]:
image_permuted = torch.permute(image, dims=(2, 0, 1))
print(f'Original shape: {image.shape}')
print(f'Shape after permuting: {image_permuted.shape}\n')
print(f'New tensor:\n {image_permuted}')

Original shape: torch.Size([1024, 1024, 3])
Shape after permuting: torch.Size([3, 1024, 1024])

New tensor:
 tensor([[[0.9683, 0.8097, 0.6498,  ..., 0.3865, 0.7619, 0.6288],
         [0.7348, 0.3168, 0.4297,  ..., 0.6849, 0.5420, 0.5327],
         [0.5963, 0.8822, 0.0236,  ..., 0.5447, 0.6046, 0.4742],
         ...,
         [0.8567, 0.4698, 0.8258,  ..., 0.7187, 0.3493, 0.0346],
         [0.6278, 0.0168, 0.8417,  ..., 0.7759, 0.0819, 0.1806],
         [0.7465, 0.6822, 0.9011,  ..., 0.5210, 0.7169, 0.2203]],

        [[0.9730, 0.8765, 0.4069,  ..., 0.8466, 0.6037, 0.7732],
         [0.9181, 0.6106, 0.4724,  ..., 0.3207, 0.3546, 0.2005],
         [0.4694, 0.2125, 0.4680,  ..., 0.1777, 0.7683, 0.0015],
         ...,
         [0.7679, 0.0566, 0.4664,  ..., 0.1043, 0.7985, 0.3198],
         [0.5091, 0.8149, 0.5538,  ..., 0.1778, 0.9297, 0.2597],
         [0.3158, 0.2986, 0.9336,  ..., 0.7761, 0.6780, 0.3433]],

        [[0.2627, 0.0550, 0.7136,  ..., 0.9223, 0.4737, 0.5016],
         [0.76